In [1]:
import os
import sys
import glob
import numpy
import pandas

# analysis
import tadpose
from tadpose import utils
from scipy.ndimage import gaussian_filter1d

# ploting
import seaborn as sns
from matplotlib import pyplot as plt



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from internals import create_experiment_table, BatchGrouper

### Create experiment table (if not done before)

In [4]:
exp_table, exp_table_path = create_experiment_table()
exp_table

Error: no input folder selected... abort


### Setup alinger

In [5]:
aligner = tadpose.alignment.TadpoleAligner(
    {"Tail_Stem": numpy.array([0, 0.0]), "Heart_Center": numpy.array([0, 1.0])},
    scale=False,
)

### BatchGrouper object

In [6]:
BG = BatchGrouper(
    exp_table="B:/fs3-bifexchgrp/BIF_StaffSci/Christoph/sweengrp/WT-Series/dataset_01/dataset_01.tab",
    aligner=aligner,
    output_grouped_by="Stage",
)

### Iterate over batch grouper object
each iteration will yield
* tad: tadpole object
* grp: name of the group
* ind: index into experiment table (usefull for writing results back)
* row_info: the row from the experiment table

In [7]:
# example

for tad, grp, ind, row_info in tqdm(BG):
    # tad: tadpole object
    # grp: name of the group
    # ind: index into experiment table
    # row_info:

    # get raw locations
    heart_pos = tad.locs(parts=("Heart_Center",)).squeeze()

    # get difference of locations (i. e. displacment)
    heart_displ = numpy.diff(heart_pos, axis=0)

    # compute the length of those displacements
    heart_speed = numpy.linalg.norm(heart_displ, axis=1)

    # smooth a little (this must be done, since we have the frame drops problems)
    heart_speed_sm = gaussian_filter1d(heart_speed, sigma=2)

    # Threshold for moving vs. not moving
    moving = heart_speed_sm > 1.5

    # compute fraction of time spent moving
    time_spent_moving = moving.sum() / len(heart_speed)

    # time spent moving
    BG.exp_table.loc[ind, "TSM"] = time_spent_moving
    # average speed while moving
    BG.exp_table.loc[ind, "ASWM"] = heart_speed_sm[moving].mean()
    # total distance
    BG.exp_table.loc[ind, "TD"] = heart_speed.sum()

NameError: name 'tqdm' is not defined

### The batch grouper will hold the table with the generated results

In [ ]:
BG.exp_table

### Plot examples on results table

In [ ]:
for feature in ["TSM", "ASWM", "TD"]:
    f, ax = plt.subplots()
    sns.barplot(y=feature, x="Stage", data=BG.exp_table, alpha=0.3, ax=ax)
    sns.stripplot(y=feature, x="Stage", data=BG.exp_table, ax=ax)
    sns.despine(ax=ax)